In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary
from torch.utils.data import DataLoader
from torchvision import datasets
import numpy as np
import random
from model_structure import get_preprocessing_transforms, train_model, evaluate_model,BCNN

In [15]:
seed = 441
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [16]:
MODEL_NUMBER = 2
BATCH_SIZE = 32
LEARNING_RATE = 0.001
NUM_EPOCHS = 60
INPUT_SIZE = 224
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [17]:
# Get transforms
train_transform, val_transform = get_preprocessing_transforms(INPUT_SIZE)

In [18]:
# master file path 
master_file_path = '/home/hsk/Documents/ML_ZK/Archive/Experiment_3_RGB_ADV'
# Set data set directory
train_dir = f'{master_file_path}/data4model_{MODEL_NUMBER}/train/'
val_dir = f'{master_file_path}/data4model_{MODEL_NUMBER}/test/'

In [19]:
# Create datasets
train_dataset = datasets.ImageFolder(
        root=train_dir,
        transform=train_transform
    )
    
val_dataset = datasets.ImageFolder(
        root=val_dir,
        transform=val_transform
    )

In [20]:
# Create data loaders
train_loader = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    num_workers=4,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False, 
    num_workers=4,
    pin_memory=True
)

In [21]:
model = BCNN(3).to(DEVICE)
criterion =  nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
model.eval()

BCNN(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Dropout2d(p=0.25, inplace=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Dropout2d(p=0.25, inplace=False)
    (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Dropout2d(p=0.25, inplace=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=100352, out_features=512, bias=True)
    (2): ReLU(inplace=True)
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=512, out_features=2, bias=True)
  )
)

In [22]:
summary(model, (3, INPUT_SIZE, INPUT_SIZE))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 224, 224]             896
              ReLU-2         [-1, 32, 224, 224]               0
         MaxPool2d-3         [-1, 32, 112, 112]               0
         Dropout2d-4         [-1, 32, 112, 112]               0
            Conv2d-5         [-1, 64, 112, 112]          18,496
              ReLU-6         [-1, 64, 112, 112]               0
         MaxPool2d-7           [-1, 64, 56, 56]               0
         Dropout2d-8           [-1, 64, 56, 56]               0
            Conv2d-9          [-1, 128, 56, 56]          73,856
             ReLU-10          [-1, 128, 56, 56]               0
        MaxPool2d-11          [-1, 128, 28, 28]               0
        Dropout2d-12          [-1, 128, 28, 28]               0
          Flatten-13               [-1, 100352]               0
           Linear-14                  [

In [23]:
tracker = train_model(model, train_loader, val_loader, criterion, optimizer, NUM_EPOCHS, DEVICE, MODEL_NUMBER)

Epoch [1/60]
Training Loss: 0.7425, Validation Loss: 0.6905
Training Acc: 0.5017, Validation Acc: 0.5000
Training F1: 0.6326, Validation F1: 0.6667
Epoch [2/60]
Training Loss: 0.6949, Validation Loss: 0.6919
Training Acc: 0.4943, Validation Acc: 0.5000
Training F1: 0.5977, Validation F1: 0.6667
Epoch [3/60]
Training Loss: 0.6966, Validation Loss: 0.6686
Training Acc: 0.5183, Validation Acc: 0.6650
Training F1: 0.6318, Validation F1: 0.6794
Epoch [4/60]
Training Loss: 0.6770, Validation Loss: 0.6329
Training Acc: 0.5790, Validation Acc: 0.6850
Training F1: 0.5691, Validation F1: 0.6702
Epoch [5/60]
Training Loss: 0.6536, Validation Loss: 0.5630
Training Acc: 0.6167, Validation Acc: 0.7300
Training F1: 0.5987, Validation F1: 0.7216
Epoch [6/60]
Training Loss: 0.6247, Validation Loss: 0.6223
Training Acc: 0.6720, Validation Acc: 0.6600
Training F1: 0.6664, Validation F1: 0.5526
Epoch [7/60]
Training Loss: 0.6037, Validation Loss: 0.5395
Training Acc: 0.6823, Validation Acc: 0.6700
Trainin